# CÁLCULO DE VIGA PROTENDIDA - VP01

**OBRA:** Carnetti Open Mall

**Cliente:** JD Construções e Pré-Fabricados

**Data:** 23/02/2022

**Cálculo:** Gabriel


Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [1]:
import handcalcs.render
import pandas as pd
from math import floor,ceil,sqrt,exp
from numpy import log

def calcpsi(tensao):
    if (tensao <= 0.5):
        return 0
    elif (tensao <= 0.6):
        extra = tensao - 0.5
        adicional = 13*extra
        return (0+adicional)/100
    elif (tensao <= 0.7):
        extra = tensao - 0.6
        adicional = 12*extra
        return (1.3+adicional)/100
    else:
        extra = tensao - 0.7
        adicional = 10*extra
        return (2.5+adicional)/100

## 1 - Informações Iniciais:

### 1.1 - Geometria da Viga:

In [2]:
%%render
# Geometria da Viga:

h = 110
b = 40
Area = 4400
Inercia = 4436667
W_i = 80667
W_s = 80667
e_p = 41
Vao = 1500

<IPython.core.display.Latex object>

### 1.2 - Propriedades dos Materiais:

In [3]:
%%render
# Propriedades dos Materiais:

# Concreto:
f_ck = 4
f_ptk = 190
f_yk = 50
f_cj = 2.8

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [4]:
%%render 2 short

E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

### 1.3 - Carregamentos:

In [5]:
%%render 2 short

#PESO PRÓPRIO DA VIGA:

Q_t0 = 0.000025*Area
Q_tF = Q_t0



<IPython.core.display.Latex object>

### 1.4 - Esforços no Centro do Vão:

In [6]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
V_F = (Q_tF)*Vao/2

<IPython.core.display.Latex object>

## 2 - Deteminação da Força de Protensão:

### 2.1 - Limites de Tensão:

In [7]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

### 2.2 - Cálculo da Força de Protensão:

In [8]:
%%render short 2

P_1 = (sigma_cc0+M_0/W_i)/(r_0*(1/Area + e_p/W_i)) 
P_2 = (sigma_ct0-M_0/W_s)/(r_0*(1/Area - e_p/W_s)) 
P_3 = (sigma_ctF+M_F/W_i)/(r_F*(1/Area + e_p/W_i))
P_4 = (sigma_ccF-M_F/W_s)/(r_F*(1/Area - e_p/W_s))


##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## 2.3 - Escolha da Protensão:

In [9]:
%%render short
# Força de Protensão Adotada:

P = 2680
phi_1 = 1.52
A_p1 = 1.40

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## 2.4 - Ancoragem das Cordoalhas:

### 2.4.1 - Comprimento de Ancoragem Básico:

In [10]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### 2.4.2 - Cálculo do comprimento de Transferência:

In [11]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.25 * l_bpt2

l_p = (h**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  3 - Esforços nas Seções:

## 3.1 - Momento Fletor e Cortante:

In [12]:
esforcos = []
for i in range(3):
    esforcos.append( [0]*20 )
    
# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(20):
    esforcos[0][i] = round(i*(Vao/19),1)
    esforcos[1][i] = round( 0.5*Q_tF*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos[2][i] = round( 0.5*Q_tF*(Vao-2*esforcos[0][i]), 1 )

In [13]:
print("\nPosição da Seção: (cm)")
print(esforcos[0],"\n")

print("Valores de M: (kNcm)")
print(esforcos[1],"\n")

print("Valores de V: (kN)")
print(esforcos[2],"\n")


Posição da Seção: (cm)
[0.0, 78.9, 157.9, 236.8, 315.8, 394.7, 473.7, 552.6, 631.6, 710.5, 789.5, 868.4, 947.4, 1026.3, 1105.3, 1184.2, 1263.2, 1342.1, 1421.1, 1500.0] 

Valores de M: (kNcm)
[0.0, 6166.9, 11655.5, 16451.9, 20568.4, 23994.4, 26738.7, 28794.3, 30166.5, 30851.7, 30851.7, 30166.5, 28794.3, 26738.7, 23994.4, 20568.4, 16451.9, 11655.5, 6166.9, 0.0] 

Valores de V: (kN)
[82.5, 73.8, 65.1, 56.5, 47.8, 39.1, 30.4, 21.7, 13.0, 4.3, -4.3, -13.0, -21.7, -30.4, -39.1, -47.8, -56.5, -65.1, -73.8, -82.5] 



# 4 - Escalonamento da Protensão:

## 4.1 - Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

In [14]:
M_s1 = esforcos[1][1]

In [15]:
%%render short

P_1 = (sigma_cc0+M_s1/W_i)/(r_0*(1/Area + e_p/W_i))
P_2 = (sigma_ct0-M_s1/W_s)/(r_0*(1/Area - e_p/W_s))


<IPython.core.display.Latex object>

In [16]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## 4.2 - Escolha do Número de Cordoalhas:

In [17]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = (8,10,12 )

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [41,41,41]

<IPython.core.display.Latex object>

# 5 - Esforços Devidos À Protensão:

In [18]:
A_p = []

for i in range(len(qntCordoalhas)):
    A_p.append(qntCordoalhas[i] * A_p1)

In [93]:
W_cp = []

for i in range(len(qntCordoalhas)):
    W_cp.append(round(Inercia/e_p_i[i],0))
    
alpha_p0 = E_p / E_cst0

K_p0 = []
for i in range(len(qntCordoalhas)):
    K_p0.append(round(alpha_p0*A_p[i]*(1/Area+e_p_i[i]/W_cp[i]),4))
    
sigma_p = []
for i in range(len(qntCordoalhas)):
    sigma_p.append(round(sigma_p0/(1+K_p0[i]),2))
    
N_p = []
for i in range(len(qntCordoalhas)):
    N_p.append(round(-sigma_p[i]*A_p[i],1))
    
M_p = []
for i in range(len(qntCordoalhas)):
    M_p.append(round(N_p[i]*e_p_i[i],1))

In [98]:
tabela1 = pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1 = tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma_p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,Sigma_p,Np,Mp
0,11.2,108211.0,0.0527,144.39,-1617.2,-66305.2
1,14.0,108211.0,0.0658,142.62,-1996.7,-81864.7
2,16.8,108211.0,0.0790,140.87,-2366.6,-97030.6


# 6 - Escalonamento da Força de Protensão:

In [108]:
M_g = []
x1 = []
x2 = []
x1_anc = []
x2_anc = []

for i in range(len(qntCordoalhas)):
    M_g.append(round( W_s*( (N_p[i]/Area) -(M_p[i]/W_s) + sigma_ct0),2) )
        
for i in range(len(qntCordoalhas)):
    #Bhaskara:
    x1.append(round( (Vao - ( Vao**2 - 8*M_g[i]/Q_t0 )**0.5)/2  ,2))
    x2.append(round( (Vao + ( Vao**2 - 8*M_g[i]/Q_t0 )**0.5)/2  ,2))
    
for i in range(len(qntCordoalhas)):
    x1_anc.append(x1[i]-round(l_bpt,2))
    x2_anc.append(x2[i]+round(l_bpt,2))

In [109]:
tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,9878.89,131.22,1368.78,-37.95,1537.95
1,18480.86,274.10,1225.90,104.93,1395.07
2,26865.23,477.89,1022.11,308.72,1191.28


# 6 - Perdas Progressivas da Força de Protensão:

## 6.1 - Fluência do Concreto:

### 6.1.1 - Fluência Rápida Irreversível

In [23]:
%%render

s = 0.38 # CP IV
t_0 = 10
t_F = 10000


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))



<IPython.core.display.Latex object>

### 6.1.2 - Fluência Lenta Irreversível:

In [24]:
%%render 2 short

U = 70 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h + 2*b )

h_fic = gamma*(2*Area)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### 6.1.3 - Fluência Lenta Reversível:

In [25]:
%%render short

phi_dF = 0.4

beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### 6.1.4 - Fluência Total:

In [26]:
%%render short
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

## 6.2 Retração do Concreto:

In [27]:
%%render short

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)


<IPython.core.display.Latex object>

## 6.3 - Relaxação do Aço de Protensão:

In [28]:
relax = []
for i in range(7):
    relax.append([0]*20)

for i in range (20):
    relax[0][i] = i+1
    relax[1][i] = esforcos[0][i] #posição x da seção
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            relax[2][i] = round(sigma_p[j],2)
            break
    relax[3][i] = round(relax[2][i]/f_ptk,2)
    relax[4][i] = round(calcpsi(relax[3][i]),2)
    relax[5][i] = round(relax[4][i]*2.5,2)
    relax[6][i] = round(-log(1-relax[5][i]),2)

In [58]:
data = pd.DataFrame(relax)
data = data.transpose()
data.rename(columns={0:'Seção',1:'X (cm)',2:'Sigma_p',3:'Sigma_p/fptk',4:'Psi1000',5:'Psi,inf',6:'Chi,inf'},inplace = True)
display(data)

,Seção,X (cm),Sigma_p,Sigma_p/fptk,Psi1000,"Psi,inf","Chi,inf"
0,1.0,0.0,144.39,0.76,0.03,0.07,0.07
1,2.0,78.9,144.39,0.76,0.03,0.07,0.07
2,3.0,157.9,142.61,0.75,0.03,0.07,0.07
3,4.0,236.8,142.61,0.75,0.03,0.07,0.07
4,5.0,315.8,140.87,0.74,0.03,0.07,0.07
5,6.0,394.7,140.87,0.74,0.03,0.07,0.07
6,7.0,473.7,140.87,0.74,0.03,0.07,0.07
7,8.0,552.6,140.87,0.74,0.03,0.07,0.07
8,9.0,631.6,140.87,0.74,0.03,0.07,0.07
9,10.0,710.5,140.87,0.74,0.03,0.07,0.07
